In [ ]:
# TODO make a list of the mushrooms names
directory = 'kaggle/sample_data/'
folder_list= [f for f in listdir(directory)]
# folder_list.remove('mushroom_sample')
# folder_list.remove('mushroom_sample.zip')
# folder_list.remove('.DS_Store')
# folder_list.remove('data_pool')
folder_list
CLASSES = dict(zip(folder_list, range(len(folder_list)))) # dictionary of the mushrooms with labels

In [ ]:
# TODO: unpickle and connect
def unpickle(directory, category ):
    """
    This functions takes the data from a directory that have been
    downsized in banches ang gives back the X,y as arrays.
    """
    table_mushrooms_downsized = []
    label_mushrooms = []
    for mushroom in CLASSES:

        dir = directory + mushroom + '/X_' + category + '_resized'
        with open(dir, "rb") as fp:  X_resized = pickle.load(fp)
        table_mushrooms_downsized.append(X_resized)
        label = np.full((1, len(X_resized)), get_label(mushroom))#.ravel()
        label_mushrooms = np.append(label_mushrooms,label)

    # flatten the nested list into a 1D list
    mushrooms_downsized = list(itertools.chain(*table_mushrooms_downsized))
    return mushrooms_downsized, label_mushrooms

    if len(mushrooms_downsized) == len(label):

    #     #Check if the number of pictures and labels are the same.
        X = np.asarray(mushrooms_downsized)
        y = np.asarray(label_mushrooms)

    return X, y
for ca in ["valid","test","train"]:
    category = ca
    X, y = unpickle(directory='/kaggle/sample_data/', category = category)
    directory = '/kaggle/working/'


    with open(directory + "X" + category, "wb") as fp: pickle.dump(X, fp)
    with open(directory + "y" + category, "wb") as fp: pickle.dump(y, fp)



In [ ]:
# shuffle the samples and labels together. Since it 
shuffler = np.random.permutation(len(X))
X = np.array(X)
y = np.array(y)
Xtrain = X[shuffler]
y= y[shuffler]
# one-hot-encode labels
ytrain = to_categorical(y)

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import os
from tensorflow.keras.models import Model
from tensorflow import keras

def model_ResNet50V2(CLASSES = CLASSES,learning_rate=0.001):
    """
    This function returns a ResNet50V2 model with the last
    layer removed.
    """
    K.clear_session() # Always clear the session!

    base_model = ResNet50V2(
        weights='imagenet', 
        pooling='avg',      # applies global average pooling to the output of the last conv layer (like a flattening)
        include_top=False,   # !!!!! we only want to have the base, not the final dense layers 
        input_shape=(224, 224, 3)  
    )
    base_model.trainable = False # To freeze the model
    # Start building on top of the model
    model = keras.Sequential() # defining a new model
    model.add(base_model) # adding in the pretrained model without the fully connected layer
    model.add(keras.layers.Dense(64, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(CLASSES), activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
    # have a look at the trainable and non-trainable params statistic
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

    return model

def fit_ResNet50V2(X, y,epochs = 200, batch_size =500,sav = True):
    """
    This function fits the model on the training data.
    """
    # observe the validation loss and stop when it does not improve after 3 iterations
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit(X, y, 
              epochs=epochs, 
              verbose=2,
              batch_size=batch_size, 
              callbacks=[callback],
              # use 30% of the data for validation
              validation_split=0.3)
    # save model
    if sav == True:
        model.save('/kaggle/working/ResNet50V2_RGB_300fot_SD.h5')
        
    return history
model = model_ResNet50V2()

history = fit_ResNet50V2(Xtrain, ytrain, epochs = 200, batch_size =300)